In [1]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.16.2


In [2]:
# Load and Normalize CIFAR-10
def normalize_images(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    return (train_images - mean) / (std + 1e-7), (test_images - mean) / (std + 1e-7)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train, x_test = normalize_images(x_train, x_test)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [3]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(x_train)


In [4]:
# Define Custom VGG16 Model Class
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = layers.Conv2D(filters, kernel_size, padding='same',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.batchnorm = layers.BatchNormalization()
        self.dropout = layers.Dropout(rate)

    def call(self, x, training=False):
        x = self.conv(x)
        x = tf.nn.relu(x)
        x = self.batchnorm(x, training=training)
        if self.drop:
            x = self.dropout(x, training=training)
        return x

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.block1 = [ConvBNRelu(64, rate=0.3), ConvBNRelu(64, drop=False), layers.MaxPooling2D()]
        self.block2 = [ConvBNRelu(128), ConvBNRelu(128, drop=False), layers.MaxPooling2D()]
        self.block3 = [ConvBNRelu(256), ConvBNRelu(256), ConvBNRelu(256, drop=False), layers.MaxPooling2D()]
        self.block4 = [ConvBNRelu(512), ConvBNRelu(512), ConvBNRelu(512, drop=False), layers.MaxPooling2D()]
        self.block5 = [ConvBNRelu(512), ConvBNRelu(512), ConvBNRelu(512, drop=False), layers.MaxPooling2D()]
        self.flatten = layers.Flatten()
        self.dropout = layers.Dropout(0.5)
        self.fc1 = layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.bn = layers.BatchNormalization()
        self.fc2 = layers.Dense(10, activation='softmax')

    def call(self, x, training=False):
        for layer in self.block1 + self.block2 + self.block3 + self.block4 + self.block5:
            x = layer(x, training=training) if isinstance(layer, ConvBNRelu) else layer(x)
        x = self.flatten(x)
        x = self.dropout(x, training=training)
        x = self.fc1(x)
        x = self.bn(x, training=training)
        x = self.fc2(x)
        return x


In [5]:
# Compile the model
vgg16.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define early stopping callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = vgg16.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

NameError: name 'vgg16' is not defined

In [ ]:
# Evaluate Custom VGG16
loss, acc = model.evaluate(x_test, y_test)
print(f"Custom VGG16 Accuracy: {acc:.4f}")


In [ ]:
#Pretrained VGG16 Transfer Learning
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)

model_pretrained = Model(inputs=base_model.input, outputs=x)
model_pretrained.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_pretrained.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test), verbose=2)

loss_pretrained, acc_pretrained = model_pretrained.evaluate(x_test, y_test)
print(f"Pretrained VGG16 Accuracy on CIFAR-10: {acc_pretrained:.4f}")
